In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [3]:
input_dir = '/content/drive/MyDrive/data/m5-data/'
target_dir = '/content/drive/MyDrive/M5-thesis/M5-FITS/processed/'
processed_data_dir_base = '/content/drive/MyDrive/M5-A1/2. data/processed/'
BASE     = processed_data_dir_base+'grid_part_1.pkl'
PRICE    = processed_data_dir_base+'grid_part_2.pkl'
CALENDAR = processed_data_dir_base+'grid_part_3.pkl'
TRAIN_START = 0

#Simplify data
Aggregate data and save it all in separate files

*   Without params
*   With params



In [4]:
global_df = pd.concat([pd.read_pickle(BASE),
                pd.read_pickle(PRICE).iloc[:,2:],
                pd.read_pickle(CALENDAR).iloc[:,2:]],
                axis=1)

In [5]:
def prepare_features(calendar):
  df = calendar.copy()
  df['snap_CA'] = df['snap_CA'].astype(int)
  df['snap_TX'] = df['snap_TX'].astype(int)
  df['snap_WI'] = df['snap_WI'].astype(int)
  df['snap_count'] = df[['snap_CA','snap_TX','snap_WI']].apply(np.sum,axis=1)

  df['is_event_1'] = [isinstance(x , str)*1 for x in df['event_name_1']]
  df['is_event_2'] = [isinstance(x , str)*1 for x in df['event_name_2']]
  df['is_event'] = df[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
  #turn into boolean
  # df['is_event'] = np.where(df['is_event']>0,1,0)

  df.drop(columns=['event_name_1', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event_1', 'is_event_2'], inplace=True)

  return df

"""
get df_train and features
"""
def get_data_by_store_id(store_id, dept_id):
  df = global_df.copy()

  store_data = df[(df['store_id'] == store_id) & (df['dept_id'] == dept_id) & (df['d'] >= TRAIN_START)]
  # df_train = store_data.pivot(index='id', columns='d', values='sales')
  # df_train.fillna(0, inplace=True)
  df_train = store_data.groupby(['d']).agg({
    'sales': 'sum',
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
    'tm_wm': 'first',
    'tm_m': 'first',
    'tm_w': 'first',
    'sell_price': 'mean',
  }).reset_index()
  # df_calendar.index = df_calendar['d']

  del df, store_data
  gc.collect()

  df_train = prepare_features(df_train)

  return df_train

In [6]:
stores = global_df['store_id'].unique()
depts = global_df['dept_id'].unique()

## Version 1
Aggregate data by stores and departments and save in separate files

In [ ]:
#VERSION 1
VERSION = 'v1'

In [ ]:
df = get_data_by_store_id('CA_1', 'HOBBIES_1')
df


,d,sales,tm_w_end,tm_dw,tm_wm,tm_m,tm_w,sell_price,snap_count,is_event
0,1,528.0,1,5,5,1,4,4.773324,0,0
1,2,489.0,1,6,5,1,4,4.773324,0,0
2,3,409.0,0,0,5,1,5,4.773324,0,0
3,4,383.0,0,1,1,2,5,4.773324,2,0
4,5,263.0,0,2,1,2,5,4.773324,2,0
...,...,...,...,...,...,...,...,...,...,...
1964,1965,0.0,0,2,3,6,24,6.629171,2,0
1965,1966,0.0,0,3,3,6,24,6.629171,0,0
1966,1967,0.0,0,4,3,6,24,6.629171,0,0
1967,1968,0.0,1,5,3,6,24,6.621959,0,0


In [ ]:
start_date = pd.Timestamp("2011-01-29")

for store in stores:
  for dept in depts:
    df = get_data_by_store_id(store, dept)
    df['date'] = start_date + pd.to_timedelta(df['d'] - 1, unit='D')
    df.to_csv(target_dir_nonparam+f'{VERSION}/data_train_{store}_{dept}.csv')

In [ ]:
aggd = global_df.groupby(['store_id', 'dept_id', 'd']).agg({
    'sales': 'sum',
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
    'tm_wm': 'first',
    'tm_m': 'first',
    'tm_w': 'first',
    'sell_price': 'mean',
  }).reset_index()

<ipython-input-9-f58533614719>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aggd = global_df.groupby(['store_id', 'dept_id', 'd']).agg({


In [ ]:
start_date = pd.Timestamp("2011-01-29")
aggd['date'] = start_date + pd.to_timedelta(aggd['d'] - 1, unit='D')
aggd.to_pickle(target_dir + '/aggregated.pkl')

In [10]:
start_date = pd.Timestamp("2011-01-29")
result = global_df[['id', 'store_id', 'dept_id', 'item_id', 'd', 'sell_price', 'sales']]
result['date'] = start_date + pd.to_timedelta(result['d'] - 1, unit='D')
result.to_pickle(target_dir + f'/raw_data.pkl')

<ipython-input-10-65a88f3ff62e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['date'] = start_date + pd.to_timedelta(result['d'] - 1, unit='D')


In [11]:
result

,id,store_id,dept_id,item_id,d,sell_price,sales,date
0,HOBBIES_1_008_CA_1_evaluation,CA_1,HOBBIES_1,HOBBIES_1_008,1,0.459961,12.0,2011-01-29
1,HOBBIES_1_009_CA_1_evaluation,CA_1,HOBBIES_1,HOBBIES_1_009,1,1.559570,2.0,2011-01-29
2,HOBBIES_1_010_CA_1_evaluation,CA_1,HOBBIES_1,HOBBIES_1_010,1,3.169922,0.0,2011-01-29
3,HOBBIES_1_012_CA_1_evaluation,CA_1,HOBBIES_1,HOBBIES_1_012,1,5.980469,0.0,2011-01-29
4,HOBBIES_1_015_CA_1_evaluation,CA_1,HOBBIES_1,HOBBIES_1_015,1,0.700195,4.0,2011-01-29
...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,WI_3,FOODS_3,FOODS_3_823,1969,2.980469,NaN,2016-06-19
47735393,FOODS_3_824_WI_3_evaluation,WI_3,FOODS_3,FOODS_3_824,1969,2.480469,NaN,2016-06-19
47735394,FOODS_3_825_WI_3_evaluation,WI_3,FOODS_3,FOODS_3_825,1969,3.980469,NaN,2016-06-19
47735395,FOODS_3_826_WI_3_evaluation,WI_3,FOODS_3,FOODS_3_826,1969,1.280273,NaN,2016-06-19
